<a href="https://colab.research.google.com/github/ProfAI/tf00/blob/master/7%20-%20Transfer%20Learning/inception.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generatori di Immagini
Spesso capita di dover addestrare una rete neurale su una mole di dati talmente grande da non riuscire a risiedere sulla memoria del nostro PC. A questo scopo tf.keras ci mette a disposizione i generatori, uno strumento che permette di caricare le immagini in batch durante la fase di addestramento.<br>
In questo notebook vedremo come utilizzare i generatori per addestrare una rete neurale in grado di distinguere 5 diversi tipi di dessert, utilizzando un dataset di 10.000 immagini.

## Scarichiamo il Dataset
Il dataset che utilizzeremo è un'estratto del Food101, una raccolta di 101.000 immagini appartenenti a 101 categorie di cibi. Nel nostr caso le categorie sono solamente le seguenti: 
 - cannoli
 - gelato
 - panna cotta
 - tiramisu
 - torta di mele

All'interno dell'archivio troviamo sia la cartella con 10.000 immagini per l'addestramento, 2000 per classe, e la cartella con 1250 immagini per il test, 250 per classe.

In [ ]:
!wget https://profession.ai/datasets/dessert.zip
!unzip -qq dessert.zip

--2020-06-29 14:14:20--  https://profession.ai/datasets/dessert.zip
Resolving profession.ai (profession.ai)... 13.225.25.70, 13.225.25.127, 13.225.25.27, ...
Connecting to profession.ai (profession.ai)|13.225.25.70|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261265207 (249M) [application/zip]
Saving to: ‘dessert.zip.1’

dessert.zip.1       100%[===================>] 249.16M  32.2MB/s    in 7.6s    

2020-06-29 14:14:27 (32.8 MB/s) - ‘dessert.zip.1’ saved [261265207/261265207]

replace dessert/test/cannoli/0003bb2524cd911b7b00f8f4c4f8bcb2.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

## Importiamo i Moduli

In [16]:
import tensorflow as tf

from tensorflow.keras import Model

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dropout, Flatten, Dense

from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Definiamo le Costanti

In [2]:
DATASET_DIR = "dessert/"
BATCH_SIZE = 64

TOT_IMAGES = 10000 
IMG_SIZE = (228, 228)

## Creiamo il Generatore

Possiamo creare il generatore utilizzando la classe *ImageDataGenerator* di tf.keras, al suo interno possiamo definire la dimensione del set di validazione.

In [3]:
datagen_train = ImageDataGenerator(validation_split=0.1, rescale=1./255,)

tramite il parametro *rescale* abbiamo definito come normalizzare le immagini, cioè dividendo per 255.<br>
Ora per creare i generatori di immagini, utilizziamo il metodo *flow_from_directory*, specificando:
 - il path al set di immagini
 - *target_size*: la dimensione da utilizzare per le immagini
 - *batch_size*: la dimensione di ogni batch di immagini
 - *class_mode*: la tipologia di classificazione, multiclasse (categorical) o binaria (binary).
 - *subset*: qui possiamo specificare se si tratta del set di training o di validazione.

In [4]:
train_generator = datagen_train.flow_from_directory(
        DATASET_DIR+"train", 
        target_size=IMG_SIZE, 
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='training')

valid_generator = datagen_train.flow_from_directory(
        DATASET_DIR+"train",
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='validation')

print(train_generator.class_indices)

Found 9000 images belonging to 5 classes.
Found 1000 images belonging to 5 classes.
{'cannoli': 0, 'gelato': 1, 'panna_cotta': 2, 'tiramisu': 3, 'torta_di_mele': 4}


Ora creiamo anche un generatore per il Test Set

In [5]:
datagen_test = ImageDataGenerator(rescale=1./255,)

test_generator = datagen_test.flow_from_directory(
        DATASET_DIR+"test",
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical')

print(test_generator.class_indices)

Found 1250 images belonging to 5 classes.
{'cannoli': 0, 'gelato': 1, 'panna_cotta': 2, 'tiramisu': 3, 'torta_di_mele': 4}


## Addestriamo la Rete Neurale Convoluzionale sul Generatore
Definiamo l'architettura della rete, trattandosi di un problema abbastanza complesso utilizzeremo diversi strati, insieme a dropout e regolarizzazione L2 per cercare di limitare l'overfiting.

In [9]:
base_model = tf.keras.applications.InceptionV3(input_shape = (228, 228, 3), include_top=False, weights='imagenet')
base_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 228, 228, 3) 0                                            
__________________________________________________________________________________________________
conv2d_188 (Conv2D)             (None, 113, 113, 32) 864         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_188 (BatchN (None, 113, 113, 32) 96          conv2d_188[0][0]                 
__________________________________________________________________________________________________
activation_188 (Activation)     (None, 113, 113, 32) 0           batch_normalization_188[0][0]    
_______________________________________________________________________________________

In [11]:
for layer in base_model.layers:
  layer.trainable = False

base_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 228, 228, 3) 0                                            
__________________________________________________________________________________________________
conv2d_188 (Conv2D)             (None, 113, 113, 32) 864         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_188 (BatchN (None, 113, 113, 32) 96          conv2d_188[0][0]                 
__________________________________________________________________________________________________
activation_188 (Activation)     (None, 113, 113, 32) 0           batch_normalization_188[0][0]    
_______________________________________________________________________________________

In [28]:
last_layer = base_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

last layer output shape:  (None, 12, 12, 768)


In [32]:
#x = tf.keras.Input(shape=(12, 12, 768))
x = Conv2D(filters=32, kernel_size=2, padding='same', activation='relu')(last_output)
x = Conv2D(filters=32, kernel_size=2, padding='same', activation='relu')(x)
x = MaxPooling2D(pool_size=3, strides=3)(x)
x = Dropout(0.5)(x)
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x)                  
x = Dense (5, activation='softmax')(x)           

model = Model(base_model.input, x) 

Per avviare l'addestramento ci basterà passare al metodo *fit* il generatore di addestramento e l'eventuale generatore di validazione. Dobbiamo solo stare attenti a specificare il numero di steps per ogni epoca del Gradient Descent, perché quando utilizziamo i generatori Tensorflow non è in grado di ottenere questo valore automaticamente, solitamente basta utilizzare: NUMERO_DI_IMMAGINI_NEL_SET/DIMENSIONE_DEL_BATCH.

In [33]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy",
                              min_delta=0.01,
                              patience=3,
                              restore_best_weights=True)

model.fit(
        train_generator,
        steps_per_epoch=int(TOT_IMAGES*0.9 // BATCH_SIZE),
        validation_data=valid_generator, 
        validation_steps=int(TOT_IMAGES*0.1 // BATCH_SIZE),
        epochs=100)

Epoch 1/100
140/140 [==============================] - 44s 315ms/step - loss: 1.0233 - accuracy: 0.6061 - val_loss: 0.7628 - val_accuracy: 0.7479
Epoch 2/100
140/140 [==============================] - 43s 309ms/step - loss: 0.6807 - accuracy: 0.7615 - val_loss: 0.6961 - val_accuracy: 0.7677
Epoch 3/100
140/140 [==============================] - 44s 313ms/step - loss: 0.6053 - accuracy: 0.7891 - val_loss: 0.6683 - val_accuracy: 0.7729
Epoch 4/100
140/140 [==============================] - 44s 311ms/step - loss: 0.5380 - accuracy: 0.8123 - val_loss: 0.6751 - val_accuracy: 0.7781
Epoch 5/100
140/140 [==============================] - 44s 315ms/step - loss: 0.4857 - accuracy: 0.8305 - val_loss: 0.7287 - val_accuracy: 0.7469
Epoch 6/100
140/140 [==============================] - 43s 309ms/step - loss: 0.4356 - accuracy: 0.8479 - val_loss: 0.7257 - val_accuracy: 0.7604
Epoch 7/100
140/140 [==============================] - 43s 308ms/step - loss: 0.3757 - accuracy: 0.8660 - val_loss: 0.7277 -

KeyboardInterrupt: ignored

# Testiamo la Rete sul Generatore
Per testare sul generatore la Rete Neurale che abbiamo addestrato, ci basta passare il generatore al metodo *evaluate*.

In [ ]:
metrics_train = model.evaluate(train_generator)
metrics_test = model.evaluate(test_generator)

print("Train Accuracy = %.4f - Train Loss = %.4f" % (metrics_train[1], metrics_train[0]))
print("Test Accuracy = %.4f - Test Loss = %.4f" % (metrics_test[1], metrics_test[0]))

 90/141 [==================>...........] - ETA: 12s - loss: 1.6128 - accuracy: 0.1924

KeyboardInterrupt: ignored